In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm
from numpy import dot
from numpy.linalg import norm
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import numpy as np
import random
from hazm import *
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
import torch
from gensim.models.keyedvectors import KeyedVectors
import numpy as np

In [ ]:
class FastText:
  def __init__(self, word_vectors, a1 , weight_type = 'simple', dims=100):
    self.weight_type = weight_type
    self.docs = a1
    self.dims = dims
    self.word_vectors = word_vectors
    self.vectorizer = TfidfVectorizer(use_idf = True, norm ='l1', ngram_range=(1,1), analyzer='word')
    self.vectorizer.fit_transform(self.docs)
    self.strings = []
    string = ''
    for item in self.vectorizer.get_feature_names():
      string += item
      string += ' '
      self.strings.append(item)
    self.vector = self.vectorizer.transform([string])
    self.vector = self.vector.toarray()
    print('vectorizer sum = ', sum(self.vector))
    # print(self.vector)
    # print(self.vector[0])
  
  def get_fasttext_average_tf_idf_wights(self,mesra):
    vector_sum = np.zeros(self.dims)
    words = mesra.split(' ')
    try:
      for word in words:
        index = self.strings.index(word)
        vector_sum += self.word_vectors[word] * self.vector[0][index]
    except Exception as e:
      pass
      # print(e.args[0])
    return vector_sum
  
  def get_fastext_average(self,mesra):
    counter = 0
    vector_sum = np.zeros(self.dims)
    words = mesra.split(' ')
    for word in words:
      try:
        vector_sum += self.word_vectors[word]
        counter += 1
      except:
        continue
      vector_sum =/ counter
    return vector_sum

  def get_fasttext(self, mesra):
    if self.weight_type == 'tfidf':
      return self.get_fasttext_average_tf_idf_wights(mesra)
    elif self.weight_type == 'simple':
      return self.get_fastext_average(mesra)

  def forward(self,query):
    ten_tokens = [0] * k
    ten_scores = [-100] * k
    vocab_error = 0
    b  = self.get_fasttext(query)

    for item in tqdm(self.docs):
      try:
        c  = self.get_fasttext(item)
        if (norm(c)*norm(b)) != 0:
          cos_sim = dot(c, b)/(norm(c)*norm(b))
          if cos_sim != 0:
            for i in range(k):
              if ten_scores[i] < cos_sim:
                ten_scores[i] = cos_sim
                ten_tokens[i] = item
                break
      except Exception as e:
        vocab_error += 1
        continue

    for i in range(len(ten_scores)):
      for j in range(len(ten_scores)):
        if ten_scores[i] > ten_scores[j]:
           ten_scores[i], ten_scores[j] = ten_scores[j], ten_scores[i]
           ten_tokens[i], ten_tokens[j] = ten_tokens[j], ten_tokens[i]

    return vocab_error, ten_tokens, ten_scores

In [ ]:
word_vectors = KeyedVectors.load_word2vec_format('vec_file.vec')
with open('sen_v2.txt', 'r') as f:
  sen_v2 = f.read()
sen_v2 = sen_v2.split('\n')
fasttext1_2 = FastText(word_vectors, sen_v2, 'simple')
words_not_found, beyts, scores = fasttext1_2.forward('کزین برتر')
